In [19]:
import scipy.io
import mne
from scipy.signal import butter, filtfilt
import os
import numpy as np
from scipy import stats

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold


from matplotlib import pyplot as plt
mne.set_log_level('error')


In [20]:
data_root = 'C:/Data/UHD_EEG/'
subjects = ['S1', 'S2', 'S3', 'S4', 'S5']
dominant_hand = ['left','right','right','right','right']

mapping = {0: "No instruction", 1: "Rest", 2: "thumb", 3: "index", 4: "middle", 5: "ring", 6: "little"}

not_ROI_channels = ['c255', 'c256', 'c254', 'c251', 'c239', 'c240', 'c238', 'c235', 'c224', 'c222', 'c223', 'c219', 'c220', 'c221', 'c215', 'c216', 'c217', 'c213', 'c212', 'c211', 'c210', 'c209', 'c112', 'c110', 'c107', 'c108', 'c103', 'c104', 'c105', 'c101', 'c100', 'c99', 'c98', 'c97', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c14', 'c15', 'c16', 'c23', 'c29', 'c26', 'c17', 'c18', 'c20', 'c19', 'c21', 'c24', 'c22', 'c25', 'c28', 'c33', 'c35', 'c38', 'c42', 'c81', 'c34', 'c37', 'c41', 'c45', 'c36', 'c40', 'c44', 'c39', 'c43', 'c145', 'c147', 'c150', 'c154', 'c157', 'c153', 'c149', 'c146', 'c93', 'c159', 'c156', 'c152', 'c148', 'c95', 'c160', 'c158', 'c155', 'c151', 'c96', 'c202', 'c198', 'c195', 'c193']

S01_bad_channels = ['c69', 'c122', 'c170', 'c173', 'c189']

In [21]:
def get_montage(hemishpere):
    mat = scipy.io.loadmat(os.path.join(data_root, 'montage', f'montage_256_{hemishpere}_hemisphere.mat'))
    return mat['pos_256']

left_handed_montage = get_montage('right')
right_handed_montage = get_montage('left')

In [22]:
def load_run(subject_id, run, describe=True):
    subject = subjects[subject_id]
    mat = scipy.io.loadmat(os.path.join(data_root, 'rawdata', subject, run))
    data = mat['y'][1:]  # remove timestamp
    ch_names = [f'c{i}' for i in range(1, 257)] + ['STIM']
    info = mne.create_info(ch_names=ch_names, sfreq=mat['SR\x00'][0][0])

    raw = mne.io.RawArray(data, info)
    ch_types = {ch: 'eeg' if ch != 'STIM' else 'stim' for ch in ch_names}
    raw.set_channel_types(ch_types)

    events = mne.find_events(raw, stim_channel='STIM')
    annot_from_events = mne.annotations_from_events(events, event_desc=mapping, sfreq=raw.info['sfreq'])
    raw.set_annotations(annot_from_events)
    raw.drop_channels(['STIM'])

    montage_positions = left_handed_montage if dominant_hand[subject_id] == 'left' else right_handed_montage
    montage = mne.channels.make_dig_montage(ch_pos=dict(zip(ch_names, montage_positions)), coord_frame='head')
    raw.set_montage(montage)

    if describe:
        raw.describe()
    return raw


In [23]:
def load_subject(subject_id, describe = True):
    runs = []
    run_files = os.listdir(os.path.join(data_root, 'rawdata', subjects[subject_id]))
    for file in run_files:
        runs.append(load_run(subject_id, file, describe))
    return runs

In [24]:
raw_runs = load_subject(subject_id = 0, describe = False)

In [25]:
preprocessed_runs = raw_runs.copy()
for run in preprocessed_runs:
    run = run.resample(200)


    run = run.drop_channels(S01_bad_channels)
    run = run.set_eeg_reference('average', projection=False)
    run = run.drop_channels(not_ROI_channels)



In [26]:
mus = []
betas = []
for i in range(len(preprocessed_runs)):
    mus.append(preprocessed_runs[i].copy().filter(l_freq=8, h_freq=12))
    betas.append(preprocessed_runs[i].copy().filter(l_freq=13, h_freq=25))

In [28]:
def centered_moving_average(data, window_size):
        half_window = window_size // 2
        cumsum = np.cumsum(data, axis=-1)
        cumsum[..., window_size:] = cumsum[..., window_size:] - cumsum[..., :-window_size]
        return (cumsum[..., window_size - 1:-window_size + 1] / window_size)


        
def my_feature(data, freq = 200, flatten = True):
    # Calculate power by squaring each time sample
    power_eeg_data = np.square(data)

    # Remove zero values to avoid log(0)
    epsilon = 1e-10
    power_eeg_data[power_eeg_data == 0] = epsilon

    # Define segment parameters
    segment_length = int(0.25 * freq)  # 0.25 s segment length 
   
   
    power_eeg_data = power_eeg_data.reshape(power_eeg_data.shape[0], -1, segment_length)
    power_eeg_data = np.mean(power_eeg_data, axis=-1)
#     print(power_eeg_data.mean(axis=1))
    power_eeg_data = centered_moving_average(power_eeg_data, 3)
#     print(power_eeg_data.mean(axis=1))


    power_eeg_data = np.clip(power_eeg_data, a_min=epsilon, a_max=None)
    power_eeg_data = np.log(power_eeg_data)
    if flatten:
        power_eeg_data = power_eeg_data.flatten()
    return power_eeg_data
    
    std = np.std(power_eeg_data, axis=1)
    mean = np.mean(power_eeg_data, axis=1)
    # print("std.shape: ", std.shape)
    # print("mean.shape: ", mean.shape)
    return np.concatenate((std, mean), axis=0) #  - 5 percent


In [33]:
middles = [] # 4
rings = []   # 5 


for i in range(len(preprocessed_runs)):
    events, event_ids = mne.events_from_annotations(preprocessed_runs[i])
    # last 25 seconds of mu
    muend = mus[i].get_data()[..., -int(25*200):]
    # last 25 seconds of beta
    betaend = betas[i].get_data()[..., -int(25*200):]

    muend = my_feature(muend, flatten=False)
    betaend = my_feature(betaend, flatten=False)
    mu_baseline = np.mean(muend, axis=-1)
    beta_baseline = np.mean(betaend, axis=-1)
    

    for trigger in events:
        if trigger[-1] in [4,5]:
            #print(trigger)
            mu_data = mus[i].get_data()[...,trigger[0]-100:trigger[0]+1400] # 0.5s after trigger to 1.5s after trigger
            mu_data -= mu_baseline[:,np.newaxis]
            
            beta_data = betas[i].get_data()[...,trigger[0]-100:trigger[0]+1400]
            beta_data -= beta_baseline[:,np.newaxis]


            mu_data = my_feature(mu_data, flatten=False)
            beta_data = my_feature(beta_data, flatten=False)

            mu_data-=mu_baseline[:,np.newaxis]
            beta_data-=beta_baseline[:,np.newaxis]

            mu_data = mu_data.flatten()
            beta_data = beta_data.flatten()

            data = np.concatenate((mu_data, beta_data), axis=0)
            
            if trigger[-1] == 4:
                middles.append(data)
            else:
                rings.append(data)
        

            
middles = np.array(middles)
rings = np.array(rings)

(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153,)
(153, 26)
(153

In [34]:
print(middles.shape)
print(rings.shape)
for ring in rings:
    print(ring.max())
    print(ring.min())
    print(ring.mean())
    break

for middle in middles:
    print(middle.max())
    print(middle.min())
    print(middle.mean())
    break

(50, 7956)
(50, 7956)
3.7219979134196155
-1.4866793741190314
0.06047677890877989
2.202524927473344
-1.421999953482315
0.1293654747667082


In [35]:
middle_vs_ring = np.concatenate((middles, rings), axis=0)

scaler = StandardScaler()
scaler.fit(middle_vs_ring)
middle_vs_ring = scaler.transform(middle_vs_ring)


y_middle_vs_ring = np.concatenate((np.zeros(middles.shape[0]), np.ones(rings.shape[0])), axis=0)


X_train, X_test, y_train, y_test = train_test_split(middle_vs_ring, y_middle_vs_ring, test_size=0.1, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


svc = SVC()
svc.fit(X_train, y_train)
svc.score(X_test, y_test)

(90, 7956)
(10, 7956)
(90,)
(10,)


0.6

In [37]:
X = middle_vs_ring
y = y_middle_vs_ring
# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf','linear'], 'gamma': [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6],
                        'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}]
grid = GridSearchCV(SVC(), tuned_parameters, cv=StratifiedKFold(n_splits=10), scoring='accuracy')
grid.fit(X, y)
print(grid.best_params_)
print(grid.best_score_)


{'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.78
